In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Specify the directory where the model is saved
load_directory = "llama3"

# Load the tokenizer and model in half precision
tokenizer = AutoTokenizer.from_pretrained(load_directory)

# Use eos_token as the pad_token
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(load_directory, torch_dtype=torch.float16).to('cuda')

# Ensure pad_token is set correctly
model.config.pad_token_id = tokenizer.pad_token_id

print("Model loaded successfully from the 'llama3' directory.")


/home/jiaping/anaconda3/envs/llama2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 13.99it/s]


Model loaded successfully from the 'llama3' directory.


In [11]:
# # Set device: GPU if available, otherwise CPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Move model to the selected device
# model = model.to(device)

# # Define the prompt template for drone interaction (focused on steps only, no backward movement)
# def get_drone_prompt(instruction, environment, obstacle):
#     prompt_template = """
#     ### Drone Instruction: {instruction}
#     ### Environment: {environment_description}
#     ### Obstacle Detected: {obstacle_type}
#     ### Drone Action (steps only, avoid the closest obstacle first, move forward and avoid moving backward):
#     """
#     return prompt_template.format(
#         instruction=instruction,
#         environment_description=environment,
#         obstacle_type=obstacle
#     )

# # Example scenario: the drone is flying in a room with obstacles
# instruction = "Fly to the opposite side of the room and avoid all obstacles."
# environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
# obstacle_type = "There is a chair 2 meters directly in front of you and a table 5 meters directly in front of you."

# # Generate the prompt
# formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type)

# # Encode the prompt using the tokenizer and move it to the same device as the model
# inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# # Ensure model is in evaluation mode and run inference
# model.eval()
# with torch.no_grad():
#     outputs = model.generate(
#         inputs,
#         max_length=256,  # Adjust the max token limit as needed
#         temperature=0.7,  # Adjust temperature for more deterministic or creative output
#         top_p=1.0,
#         top_k=0,  # Disable top-k filtering by setting top_k to 0
#         repetition_penalty=1.0,
#         early_stopping=True,
#         pad_token_id=tokenizer.eos_token_id  # Set pad token ID to avoid warnings
#     )

# # Decode the generated output back to text
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Display the drone's action based on the generated response (steps only)
# print("Drone's response to obstacle (steps only):")
# print(generated_text)


Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 2 meters directly in front of you and a table 5 meters directly in front of you.
    ### Drone Action (steps only, avoid the closest obstacle first, move forward and avoid moving backward):
     - Move 0.5 meters forward (avoid the chair)
     - Move 0.5 meters forward (move past the chair)
     - Move 0.5 meters forward (avoid the table)
     - Move 0.25 meters forward (move past the table)
     - Move 0.25 meters forward (move past the table)
     - Move 0.25 meters forward (move past the table)
     - Move 0.25 meters forward (move past the table)
     - Move 0.25 meters forward (move past the table)
     - Move 0.25 meters forward (move past the table)
     - Move 0.25 meters forward (move past the table)
    ### D

In [23]:
# current best

def get_drone_prompt(instruction, environment, obstacle, target):
    prompt_template = """
    ### Drone Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle Detected: {obstacle_type}
    ### Target Location: {target_location}
    ### Drone Action: 
    Navigate towards the target using the following step-by-step instructions. 
    - Avoid all obstacles in the path by side stepping
    - Move forward to reach the target location without moving backward
    - Be concise in direction changes (e.g., turn left/right 90 degrees)
    - Stop and side step if the obstacle is 1 meter directly in front of you
    
    Follow these steps:
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle,
        target_location=target
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "There is a chair 3 meters directly in front of you."
target_location = "Opposite side of the room (15 meters away)."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type, target_location)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=300,  # Increase the max token limit for a longer response
        temperature=0.4,  # Lower temperature for more deterministic responses
        top_p=0.95,       # Adjust to a broader probability range
        top_k=50,         # Consider more tokens for diversity in navigation
        repetition_penalty=1.15,  # Slight penalty to avoid repetitive text
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's action based on the generated response (steps only)
print("Drone's response to obstacle (steps only):")
print(generated_text)


Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 3 meters directly in front of you.
    ### Target Location: Opposite side of the room (15 meters away).
    ### Drone Action: 
    Navigate towards the target using the following step-by-step instructions. 
    - Avoid all obstacles in the path by side stepping
    - Move forward to reach the target location without moving backward
    - Be concise in direction changes (e.g., turn left/right 90 degrees)
    - Stop and side step if the obstacle is 1 meter directly in front of you
    
    Follow these steps:
     1. Turn right 45 degrees to move around the chair.
     2. Side step 0.5 meters to maintain distance from the chair.
     3. Move forward 4 meters.
     4. Turn left 135 degrees to align with the target directi

In [12]:
# still testing

# Set device: GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the selected device
model = model.to(device)

# Define the prompt template for drone interaction (focused on steps only, no backward movement)
def get_drone_prompt(instruction, environment, obstacle, target):
    prompt_template = """
    ### Drone Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle Detected: {obstacle_type}
    ### Target Location: {target_location}
    ### Drone Action (navigate towards the target, avoid the closest obstacle first, use sidesteps to avoid any obstacles, move forward when there is no obstacle in, clear and concise, focus on avoiding obstacles and moving forward):
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle,
        target_location=target
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "There is a chair 3 meters directly in front of you and a table 4 meters in front of you."
target_location = "Opposite side of the room (15 meters away)."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type, target_location)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=256,  # Adjust the max token limit as needed
        temperature=0.5,  # Lower temperature for more deterministic responses
        top_p=0.9,        # Narrow down the probability range
        top_k=40,         # Choose from the top 40 most likely next tokens
        repetition_penalty=1.1,  # Slight penalty to avoid repetitive text
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's action based on the generated response (steps only)
print("Drone's response to obstacle (steps only):")
print(generated_text)

Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 3 meters directly in front of you and a table 4 meters in front of you.
    ### Target Location: Opposite side of the room (15 meters away).
    ### Drone Action (navigate towards the target, avoid the closest obstacle first, use sidesteps to avoid any obstacles, move forward when there is no obstacle in, clear and concise, focus on avoiding obstacles and moving forward):
     - Move forward 5 meters to create space between me and the chair.
     - Sidestep left by 2 meters to avoid the chair.
     - Move forward 4 meters to approach the table.
     - Sidestep right by 1 meter to avoid the table.
     - Move forward 6 meters to reach the middle of the room.
     - Continue moving forward until I reach the opposite side

In [18]:
def get_drone_prompt(instruction, environment, obstacle, target):
    prompt_template = """
    ### Drone Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle Detected: {obstacle_type}
    ### Target Location: {target_location}
    ### Drone Action: 
    Navigate towards the target using the following step-by-step instructions. 
    - Focus on sidestepping obstacles.
    - Move forward when clear, and turn as needed to avoid obstacles.
    - Be concise in direction changes (e.g., turn left/right 90 degrees).
    
    Follow these steps:
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle,
        target_location=target
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "There is a chair 3 meters directly in front of you and a table 4 meters in front of you."
target_location = "Opposite side of the room (15 meters away)."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type, target_location)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=256,  # Adjust the max token limit as needed
        temperature=0.5,  # Lower temperature for more deterministic responses
        top_p=0.9,        # Narrow down the probability range
        top_k=40,         # Choose from the top 40 most likely next tokens
        repetition_penalty=1.1,  # Slight penalty to avoid repetitive text
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's action based on the generated response (steps only)
print("Drone's response to obstacle (steps only):")
print(generated_text)


Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 3 meters directly in front of you and a table 4 meters in front of you.
    ### Target Location: Opposite side of the room (15 meters away).
    ### Drone Action: 
    Navigate towards the target using the following step-by-step instructions. 
    - Focus on sidestepping obstacles.
    - Move forward when clear, and turn as needed to avoid obstacles.
    - Be concise in direction changes (e.g., turn left/right 90 degrees).
    
    Follow these steps:
     1. Move forward until you're 2 meters from the chair.
     2. Turn right 45 degrees to move around the chair.
     3. Move forward until you're 2 meters past the chair.
     4. Turn right 90 degrees to change course and avoid the table.
     5. Move forward until you

In [7]:
# # Define the prompt template for drone interaction (focused on steps only, no backward movement)
def get_drone_prompt(instruction, environment, obstacle, target):
    prompt_template = """
    ### Drone Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle Detected: {obstacle_type}
    ### Target Location: {target_location}
    ### Drone Action (navigate towards the target, avoid the closest obstacle first, use sidesteps to avoid obstacles, and move forward in small steps):
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle,
        target_location=target
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "There is a chair 3 meters directly in front of you and a table 4 meters in front of you."
target_location = "Opposite side of the room (15 meters away)."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type, target_location)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=256,  # Adjust the max token limit as needed
        temperature=0.5,  # Lower temperature for more deterministic responses
        top_p=0.9,        # Narrow down the probability range
        top_k=40,         # Choose from the top 40 most likely next tokens
        repetition_penalty=1.1,  # Slight penalty to avoid repetitive text
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's action based on the generated response (steps only)
print("Drone's response to obstacle (steps only):")
print(generated_text)

Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 3 meters directly in front of you and a table 4 meters in front of you.
    ### Target Location: Opposite side of the room (15 meters away).
    ### Drone Action (navigate towards the target, avoid the closest obstacle first, use sidesteps to avoid obstacles, and move forward in small steps):
     - Move forward by 1 meter
     - Check if there's an obstacle ahead
     - If there's an obstacle, calculate its distance from the drone
     - If the obstacle is closer than the target, sidestep to the left/right by 0.5 meters
     - Repeat the process until reaching the target

```python
import math

# Define the environment dimensions
ROOM_WIDTH = 15
ROOM_HEIGHT = 15

# Define the obstacle positions
CHAIR_X = 7
CHAIR_Y = 1

In [6]:
import math

# Define the initial position and orientation of the drone
drone_x = 0
drone_y = 0
drone_orientation = 0  # Assuming 0 degrees is facing "up" or forward

# Function to update the drone's position based on a step command
def update_drone_position(drone_x, drone_y, drone_orientation, movement_type, distance):
    if movement_type == "forward":
        # Move in the direction the drone is facing
        drone_x += distance * math.cos(math.radians(drone_orientation))
        drone_y += distance * math.sin(math.radians(drone_orientation))
    elif movement_type == "sidestep left":
        # Move perpendicular to the current direction (left side is -90 degrees)
        drone_x += distance * math.cos(math.radians(drone_orientation - 90))
        drone_y += distance * math.sin(math.radians(drone_orientation - 90))
    elif movement_type == "sidestep right":
        # Move perpendicular to the current direction (right side is +90 degrees)
        drone_x += distance * math.cos(math.radians(drone_orientation + 90))
        drone_y += distance * math.sin(math.radians(drone_orientation + 90))
    
    return drone_x, drone_y

# Example generated drone instructions
instructions = [
    ("forward", 2),  # Move 2 meters forward
    ("sidestep left", 1),  # Sidestep left by 1 meter
    ("forward", 2),  # Move forward again by 2 meters
    ("sidestep right", 0.5),  # Sidestep right by 0.5 meters
    ("forward", 10)  # Move forward until reaching the opposite side
]

# Simulate the drone's movement step by step
for step in instructions:
    movement_type, distance = step
    drone_x, drone_y = update_drone_position(drone_x, drone_y, drone_orientation, movement_type, distance)
    print(f"Drone's new position after {movement_type} by {distance} meters: (x: {drone_x:.2f}, y: {drone_y:.2f})")

# Final position
print(f"Final drone position: (x: {drone_x:.2f}, y: {drone_y:.2f})")


Drone's new position after forward by 2 meters: (x: 2.00, y: 0.00)
Drone's new position after sidestep left by 1 meters: (x: 2.00, y: -1.00)
Drone's new position after forward by 2 meters: (x: 4.00, y: -1.00)
Drone's new position after sidestep right by 0.5 meters: (x: 4.00, y: -0.50)
Drone's new position after forward by 10 meters: (x: 14.00, y: -0.50)
Final drone position: (x: 14.00, y: -0.50)


In [10]:
# Define the prompt template for drone interaction (focused on steps only, no backward movement)
def get_drone_prompt(instruction, environment, obstacle, target):
    prompt_template = """
    ### Drone Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle Detected: {obstacle_type}
    ### Target Location: {target_location}
    ### Drone Action (navigate towards the target, avoid the closest obstacle first, use sidesteps to avoid obstacles, and move forward in small steps):
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle,
        target_location=target
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "There is a chair 3 meters directly in front of you and a table 4 meters in front of you."
target_location = "Opposite side of the room (15 meters away)."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type, target_location)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=256,  # Adjust the max token limit as needed
        temperature=0.5,  # Lower temperature for more deterministic responses
        top_p=0.9,        # Narrow down the probability range
        top_k=40,         # Choose from the top 40 most likely next tokens
        repetition_penalty=1.1,  # Slight penalty to avoid repetitive text
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's action based on the generated response (steps only)
print("Drone's response to obstacle (steps only):")
print(generated_text)

Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 3 meters directly in front of you and a table 4 meters in front of you.
    ### Target Location: Opposite side of the room (15 meters away).
    ### Drone Action (navigate towards the target, avoid the closest obstacle first, use sidesteps to avoid obstacles, and move forward in small steps):
     - Move forward by 1 meter.
     - Detect obstacles within 2 meters radius.
     - If an obstacle is detected:
       - Calculate distance to each obstacle.
       - Choose the closest obstacle and calculate its angle relative to your current position.
       - Turn left or right based on the calculated angle.
       - Repeat this process until the obstacle is avoided.

### Code

```python
import math

class Drone:
    def __i

In [3]:
# Set device: GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the selected device
model = model.to(device)

# Define the prompt template for drone interaction (focused on steps only, no backward movement)
def get_drone_prompt(instruction, environment, obstacle):
    prompt_template = """
    ### Drone Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle Detected: {obstacle_type}
    ### Drone Action (steps only, avoid the closest obstacle first, move forward and avoid moving backward):
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "There is a chair 2 meters directly in front of you and a table 5 meters directly in front of you."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=256,  # Adjust the max token limit as needed
        temperature=0.7,  # Adjust temperature for more deterministic or creative output
        top_p=1.0,
        top_k=0,  # Disable top-k filtering by setting top_k to 0
        repetition_penalty=1.0,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id  # Set pad token ID to avoid warnings
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's action based on the generated response (steps only)
print("Drone's response to obstacle (steps only):")
print(generated_text)

/home/jiaping/anaconda3/envs/llama2/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 2 meters directly in front of you and a table 5 meters directly in front of you.
    ### Drone Action (steps only, avoid the closest obstacle first, move forward and avoid moving backward):
     - Move forward 5 meters.
     - Turn 90 degrees to the left and move forward 5 meters.
     - Turn 90 degrees to the left and move forward 5 meters.
     - Turn 90 degrees to the left and move forward 1 meter.
    ### Question: What is the drone's final position and orientation after executing these instructions?

# Drone Instruction: Fly to the opposite side of the room and avoid all obstacles.
# Environment: The room is 15x15 meters with several tables and chairs scattered around.
# Obstacle Detected: There is a chair 2 meter

In [9]:
# Set device: GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the selected device
model = model.to(device)

# Define the prompt template for drone interaction
def get_drone_prompt(instruction, environment, obstacle):
    prompt_template = """
    ### Drone Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle Detected: {obstacle_type}
    ### Drone Action (steps only):
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly forward and avoid obstacles."
environment_description = "The room is 10x10 meters with several tables and chairs scattered around."
obstacle_type = "There is a chair 5 meters in front and a table 2 meters infront."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=256,  # Adjust the max token limit as needed
        temperature=0.7,  # Adjust temperature for more deterministic or creative output
        top_p=1.0,
        top_k=0,  # Disable top-k filtering by setting top_k to 0
        repetition_penalty=1.0,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id  # Set pad token ID to avoid warnings
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's action based on the generated response
print("Drone's response to obstacle (steps only):")
print(generated_text)

Drone's response to obstacle (steps only):

    ### Drone Instruction: Fly forward and avoid obstacles.
    ### Environment: The room is 10x10 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 5 meters in front and a table 2 meters infront.
    ### Drone Action (steps only):
     1. Fly forward 3 meters.
     2. Obstacle detected: chair 5 meters in front.
     3. Obstacle detected: table 2 meters in front.
     4. Fly forward 1 meter.
     5. Obstacle detected: table 2 meters in front.
     6. Fly forward 1.5 meters.
     7. No obstacles detected.
    ### Drone Action is: Return to base.

The final answer is: Return to base.     ### Drone Instruction: Fly forward and avoid obstacles.
    ### Environment: The room is 10x10 meters with several tables and chairs scattered around.
    ### Obstacle Detected: There is a chair 5 meters in front and a table 2 meters infront.
    ### Drone Action (steps only):
     1. Fly forward 3 meters.
     

In [12]:
# Define the refined prompt template for drone navigation (focused on concise and efficient steps)
def get_drone_prompt(instruction, environment, obstacle, target):
    prompt_template = """
    ### Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle: {obstacle_type}
    ### Target: {target_location}
    ### Navigation Instructions (clear and concise, focus on avoiding obstacles and moving forward):
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle,
        target_location=target
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "A chair is 2 meters in front, a table is 4 meters ahead, and other scattered items."
target_location = "The opposite side of the room (15 meters away)."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type, target_location)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=150,  # Shorter max token limit to focus on efficiency
        temperature=0.3,  # Lower temperature for more deterministic responses
        top_p=0.85,       # Lower value to focus on the top-ranked tokens
        top_k=30,         # Fewer top choices for more direct actions
        repetition_penalty=1.05,  # Mild repetition penalty to prevent redundancy
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's concise navigation instructions
print("Drone's response (navigation steps only):")
print(generated_text)

Drone's response (navigation steps only):

    ### Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle: A chair is 2 meters in front, a table is 4 meters ahead, and other scattered items.
    ### Target: The opposite side of the room (15 meters away).
    ### Navigation Instructions (clear and concise, focus on avoiding obstacles and moving forward):
     - Move forward 5 meters.
     - Turn left 90 degrees.
     - Move forward 3 meters.
     - Turn right 90 degrees.
     - Move forward 2 meters.
     - Turn left 90 degrees.
     - Move forward


In [28]:

# Define the refined prompt template for drone navigation (focused on concise and efficient steps)
def get_drone_prompt(instruction, environment, obstacle, target):
    prompt_template = """
    ### Instruction: {instruction}
    ### Environment: {environment_description}
    ### Obstacle: {obstacle_type}
    ### Target: {target_location}
    ### Navigation Instructions (number the steps, focus on avoiding obstacles and moving forward efficiently):
    """
    return prompt_template.format(
        instruction=instruction,
        environment_description=environment,
        obstacle_type=obstacle,
        target_location=target
    )

# Example scenario: the drone is flying in a room with obstacles
instruction = "Fly to the opposite side of the room and avoid all obstacles."
environment_description = "The room is 15x15 meters with several tables and chairs scattered around."
obstacle_type = "A chair is 2 meters in front, a table is 4 meters ahead, and other scattered items."
target_location = "The opposite side of the room (15 meters away)."

# Generate the prompt
formatted_prompt = get_drone_prompt(instruction, environment_description, obstacle_type, target_location)

# Encode the prompt using the tokenizer and move it to the same device as the model
inputs = tokenizer.encode(formatted_prompt, return_tensors="pt").to(device)

# Ensure model is in evaluation mode and run inference
model.eval()
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_length=200,  # Increased max token limit to allow complete response
        temperature=0.2,  # Lower temperature for more deterministic responses
        top_p=0.9,        # Keep top_p to ensure variety but avoid outliers
        top_k=40,         # Top 40 tokens to balance between variety and accuracy
        repetition_penalty=1.2,  # Increase repetition penalty to avoid repeating unnecessary steps
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )

# Decode the generated output back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Display the drone's concise navigation instructions
print("Drone's response (navigation steps only):")
print(generated_text)

Drone's response (navigation steps only):

    ### Instruction: Fly to the opposite side of the room and avoid all obstacles.
    ### Environment: The room is 15x15 meters with several tables and chairs scattered around.
    ### Obstacle: A chair is 2 meters in front, a table is 4 meters ahead, and other scattered items.
    ### Target: The opposite side of the room (15 meters away).
    ### Navigation Instructions (number the steps, focus on avoiding obstacles and moving forward efficiently):
     - Step 1: Move straight for 5 meters while keeping an eye out for any objects that might be blocking your path. 
     - Step 2: If you see a chair or another obstacle within this distance, adjust your course slightly left or right by about one meter to go around it without touching anything. Otherwise, continue moving straight until you reach the end of step 3.
     - Step 3: Once past the initial area where potential obstructions were present, move directly towards
